In [65]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores.pinecone import Pinecone
#
from domino_data.vectordb import DominoPineconeConfiguration
from mlflow.deployments import get_deploy_client
from langchain_community.embeddings import MlflowEmbeddings
from langchain_community.chat_models import ChatMlflow

import pinecone
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [66]:
#os.environ['ANTHROPIC_API_KEY'] = getpass("Enter Anthropic key:")
#os.environ['OPENAI_API_KEY'] = getpass("Enter OpenAI API key:")
#os.environ['PINECONE_API_KEY'] = getpass("Enter Pinecone API key:")
#os.environ['PINECONE_ENV'] = getpass("Enter Pinecone Environment:")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_API_ENV')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [67]:
# Create embeddings to embed queries
embed = MlflowEmbeddings(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="embedding-ada-002ja",
)

In [68]:
text_field = "symptoms"

datasource_name = "MedRAG-JA"
conf = DominoPineconeConfiguration(datasource=datasource_name)

#api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)
api_key = PINECONE_API_KEY
# initialize pinecone
pinecone.init(
    api_key=api_key,
    environment=PINECONE_ENV,
   # openapi_config=conf
)

index_name = "medical-qa-search"
index = pinecone.Index(index_name)

# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

In [69]:
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [70]:
rag_llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4',
    temperature=0.0
)

In [71]:
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [72]:
user_question = input("Please provide the symptoms here :")
result = qa_chain(user_question)

Please provide the symptoms here : What are the symptoms of shingles


In [73]:
result['result']

'The symptoms of shingles include pain or a tingling sensation in a limited area on one side of the face or torso, followed by a red rash with small, fluid-filled blisters. Other symptoms may include pain, burning, numbness or tingling, sensitivity to touch, a red rash that begins a few days after the pain, fluid-filled blisters that break open and crust over, itching, fever, headache, sensitivity to light, and fatigue. The pain and rash usually occur near an eye and if left untreated, this infection can lead to permanent eye damage.'

In [43]:
result['source_documents'][0].page_content

'[\'\', \'Shingles is characterized by pain or a tingling sensation in a limited area on one side of the face or torso, followed by a red rash with small, fluid-filled blisters.\', \'\', \'The signs and symptoms of shingles usually affect only a small section of one side of your body. These signs and symptoms may include:\', \'Some people also experience:\', \'Pain is usually the first symptom of shingles. For some, it can be intense. Depending on the location of the pain, it can sometimes be mistaken for a symptom of problems affecting the heart, lungs or kidneys. Some people experience shingles pain without ever developing the rash.\', \'Most commonly, the shingles rash develops as a stripe of blisters that wraps around either the left or right side of your torso. Sometimes the shingles rash occurs around one eye or on one side of the neck or face.\', \'Contact your doctor promptly if you suspect shingles, but especially in the following situations:\', \'\', \'\', \'\', \'\', \'\', \